# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [3]:
import pandas as pd

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
data = pd.read_csv('/Users/alexandradolidze/Desktop/compling/labeled.csv')

In [6]:
data.toxic.value_counts(normalize=True)

0.0    0.66514
1.0    0.33486
Name: toxic, dtype: float64

In [7]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [8]:
y_train = train.toxic.values
y_test = test.toxic.values

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


from razdel import tokenize

In [10]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 

In [11]:
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred, digits = 4))

              precision    recall  f1-score   support

         0.0     0.8577    0.9301    0.8924       959
         1.0     0.8333    0.6936    0.7571       483

    accuracy                         0.8509      1442
   macro avg     0.8455    0.8119    0.8248      1442
weighted avg     0.8495    0.8509    0.8471      1442



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


С токенизацией razdel

In [12]:
vectorizer = CountVectorizer(tokenizer=lambda x: [w.text for w in tokenize(x)])
X_train_razdel = vectorizer.fit_transform(train.comment)
X_test_razdel = vectorizer.transform(test.comment)


In [13]:
model_razdel = LogisticRegression()
model_razdel.fit(X_train_razdel, y_train)

y_pred_razdel = model_razdel.predict(X_test_razdel)

print(classification_report(y_test, y_pred_razdel, digits = 4))

              precision    recall  f1-score   support

         0.0     0.8688    0.9249    0.8960       959
         1.0     0.8290    0.7226    0.7721       483

    accuracy                         0.8571      1442
   macro avg     0.8489    0.8237    0.8340      1442
weighted avg     0.8554    0.8571    0.8545      1442



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Модель, обучавшаяся на данных, токенезированных razdel, дает немного (совсем немного) лучшие результаты, показывая более высокие f1-score и accuracy

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [14]:
import re, string, math
import numpy as np

In [15]:
sentences = ['я и ты', 'ты и я', 'я, я и только я', 'только не я', 'он']

In [16]:
word_set = [re.findall(r'\w+', sentence) for sentence in sentences]
total_documents = len(sentences)
unique_words = list(set(i for j in word_set for i in j))

print(total_documents, unique_words)

5 ['я', 'он', 'только', 'ты', 'не', 'и']


In [17]:
each_sentence = []
idf = {}
for sentence in word_set:
    tf = {}
    for token in sentence:
        if token in tf.keys():
            tf[token] +=1
        else:
            tf[token] = 1
            if token in idf.keys():
                idf[token] +=1
            else:
                idf[token] = 1
                
    for item in tf.keys():
        tf[item] = tf[item]/len(sentence)

    each_sentence.append(tf)

for i in idf.keys():
    idf[i] = (math.log(len(sentences)/idf[i]))

In [18]:
results_df = pd.DataFrame(0.0, columns = unique_words, index = sentences)

In [19]:
for row, sentence in enumerate(each_sentence):
    for word in sentence.keys():
        for k in idf.keys():
            if word == k:
                tf = sentence[word]
#                 print([word, row],tf, idf[word])
                results_df[word][row] = tf * idf[word]

In [20]:
results_df

,я,он,только,ты,не,и
я и ты,0.074381,0.000000,0.000000,0.30543,0.000000,0.170275
ты и я,0.074381,0.000000,0.000000,0.30543,0.000000,0.170275
"я, я и только я",0.133886,0.000000,0.183258,0.00000,0.000000,0.102165
только не я,0.074381,0.000000,0.305430,0.00000,0.536479,0.000000
он,0.000000,1.609438,0.000000,0.00000,0.000000,0.000000


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [21]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split

In [22]:
data = pd.read_csv('/Users/alexandradolidze/Desktop/compling/labeled.csv')

In [23]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

y_train = train.toxic.values
y_test = test.toxic.values

In [24]:
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [28]:
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
stopwords = stopwords.words('russian')

print(stopwords)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexandradolidze/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [62]:
vectorizer1 = CountVectorizer(lowercase = False, binary = True, stop_words = stopwords, max_df = 0.9, min_df = 5, max_features = 10000, ngram_range=(1,3))
vectorizer2 = TfidfVectorizer(stop_words = stopwords, binary = True, max_df = 0.9, min_df = 5, max_features = 10000, ngram_range=(1,3))

In [96]:
# LogisticRegression 
X_train1 = vectorizer1.fit_transform(train.comment)
X_test1 = vectorizer1.transform(test.comment) 

classifier1 = LogisticRegression(solver = 'liblinear', penalty = 'l2', class_weight = 'balanced')
classifier1.fit(X_train1, y_train)

y_preds1 = classifier1.predict(X_test1)

print(classification_report(y_test, y_preds1))

              precision    recall  f1-score   support

         0.0       0.89      0.81      0.85       970
         1.0       0.67      0.80      0.73       472

    accuracy                           0.81      1442
   macro avg       0.78      0.80      0.79      1442
weighted avg       0.82      0.81      0.81      1442



In [88]:
# DecisionTreeClassifier
X_train2 = vectorizer2.fit_transform(train.comment)
X_test2 = vectorizer2.transform(test.comment) 

сlassifier2 = DecisionTreeClassifier(criterion = 'log_loss', splitter = 'random', max_depth = 1500, min_samples_split = 10, class_weight = 'balanced', random_state = 322)
сlassifier2.fit(X_train2, y_train)

y_preds2 = сlassifier2.predict(X_test2)

print(classification_report(y_test, y_preds2))

              precision    recall  f1-score   support

         0.0       0.84      0.81      0.82       970
         1.0       0.63      0.69      0.66       472

    accuracy                           0.77      1442
   macro avg       0.74      0.75      0.74      1442
weighted avg       0.77      0.77      0.77      1442



In [98]:
# индексы с самой высокой вероятностью в классе токсичных (toxic = 1.0)

y_probas1 = classifier1.predict_proba(X_test1)
y_probas2 = сlassifier2.predict_proba(X_test2)

# np.argsort(y_probas1, axis=0)[:10][:,0]

In [124]:
toxic_texts = pd.DataFrame(columns = ['LogisticRegression', 'DecisionTreeClassifier'])

In [133]:
# Самые токсичные тексты у первого классификатора
toxic_texts['LogisticRegression'] = test.loc[np.argsort(y_probas1, axis=0)[:10][:,0],'comment'].values
test.loc[np.argsort(y_probas1, axis=0)[:10][:,0],'comment'].values

array(['Пиздец у быдла с пикабу сначала горело от негров на нулевой, теперь от скримеров, куда я нахуй попал, ебаные животные это БЭ, ЭТО РАНДОМ СУЧАРА, ТАМ НЕ ДОЛЖНО БЫТЬ ПРАВИЛ, ПОШЕЛ НАХУЙ\n',
       'Да, Эстетам похуй . Когда ваш главэстет будит униженно на весь интернет опровергать свой пиздёж про старикашку - вам тоже похуй будет? Ты долбоёб, гоблядь, Маэстро если и сделает такое извенение , то в нём ещё сильнее обосрёт старикашку и хайпанёт на этом)) Можешь придумывать кликбэйтные заголовки и отправлять лакеям)) Величайший историк извеняется перед гоблинским подсосом)) Маэстро: По решению подзалупного суда косплеер не плагиатор, не бандит, не насильник, не педофил, слышите!? И сам пишет свои маняграфии)) Пусть до конца жизни судится, спляшем с Книгой Века на его могиле))',
       'Стас, никому, кроме тебя и армии твоих подсосов(которые представляют собой типичный дегенеративный биомусор, ведущийся на любые скандалы-интриги), твои ролики нахуй не нужны. Серьёзно, ты сделал новост

In [134]:
# Самые токсичные тексты у второго классификатора
toxic_texts['DecisionTreeClassifier'] = test.loc[np.argsort(y_probas2, axis=0)[:10][:,0],'comment'].values
test.loc[np.argsort(y_probas2, axis=0)[:10][:,0],'comment'].values

array(['Сука тупой дегенарт. Вот на эти видео. Съеби уже к своему старому куколду в жж, жухлые сморчки друг-другу теребить.\n',
       'Верните мне нормальную советскую медицину, я жить хочу.\n',
       'Кредитнулся и взял телефон Samsung WiTu Omnia. Поставил цифровой пароль и не прошло и месяца как телефон перестал узнавать пароль. Звонил в техподдержку Samsung ,а я и слово не успел сказать мне говорят ниипёт это не гарантийный случай п...уй в сервисный центр Samsung. В общем разблокировали мне его за 1000 причём попросили подписать бумагу если чо то притензий не имею. Из-за всего этого у меня остался какой-то неприятный осадочек. С тех пор телефоны и технику этой компании обходу стороной. Через 10 лет решил купить себе iPhone 5s чтобы узнать как это быть владельцем яблофона. Когда возникла проблема с телефоном я позвонил в поддержку Apple и мне там охотно помогли. С тех пор при всех минусах iPhone я более лоялен к этой технике. Знаю что отдав деньги как за 2 Samsunga меня не пошлют н

In [128]:
toxic_texts

,LogisticRegression,DecisionTreeClassifier
0,Пиздец у быдла с пикабу сначала горело от негр...,Сука тупой дегенарт. Вот на эти видео. Съеби у...
1,"Да, Эстетам похуй . Когда ваш главэстет будит ...","Верните мне нормальную советскую медицину, я ж..."
2,"Стас, никому, кроме тебя и армии твоих подсосо...",Кредитнулся и взял телефон Samsung WiTu Omnia....
3,"Двач. Ну это пиздец, конечно. Раньше я его ува...",Могут ли они пережить атомной взрыв? Если сост...
4,"В Киеве на вокзале Мен було рок в 19, коли мен...","почему? Логика следующая - в фокусе центр,а че..."
5,"О, свиньи вздумали еще и пидорах перефорсить. ...",Я просто общаюсь Пидораха пидораха Скотоублюди...
6,Западенские каклы в Польше - это чмо пиздец ка...,"Из обсуждения в интернете чей Крым, ничего не ..."
7,"БЕЛАРУСЬ, БЛЯТЬ, БЕЛАРУСЬ. СПИДОРСВИН, БЛЯТЬ. ...",Как подробно он о себе рассказал все-таки.\n
8,"залетуха плиз, если ты так любишь покой, то ты...","Русская кореец (на второй фотке мамка, да)\n"
9,"Какие же хохлы незалежные дегенераты, пиздец п...",А почему ты упустил вот это Ты не упустил. Мол...


Как-то они совершенно не совпадают.

Видно, что дерево решений обозначило токсичными совсем не (или практически не) токсичные высказываний (например, №1-4), которые содержат ненормативную лексику, однако она явно используется с целью усилить эмоцию, передаваемую автором, а не оскорбить оппонента. 

В то же время классификатор на основе логистической регрессии показывает всю силу ненависти пользователей двача друг к другу и к самим себе. Интересно, что как токсичный был корректно классифицирован текст на украинском языке, который вероятно попал в датасет по ошибке при разметке.

## *Задание 4 (2 балла)

Для классификаторов LogisticRegression и Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [135]:
data = pd.read_csv('/Users/alexandradolidze/Desktop/compling/labeled.csv')

In [136]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

y_train = train.toxic.values
y_test = test.toxic.values

In [138]:
# буду работать с CountVectorizer с гиперпараметрами как в предыдудщем задании
vectorizer = CountVectorizer(lowercase = False, stop_words = stopwords, max_df = 0.9, min_df = 5, max_features = 10000, ngram_range=(1,3))

X_train = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 

In [213]:
сlassifier1 = LogisticRegression()
сlassifier1.fit(X_train, y_train)

y_preds1 = сlassifier1.predict(X_test)

coeffs1 = сlassifier1.coef_

In [214]:
сlassifier2 = RandomForestClassifier(random_state = 322)
сlassifier2.fit(X_train, y_train)

y_preds2 = сlassifier2.predict(X_test)

coeffs2 = сlassifier2.feature_importances_

In [216]:
np.argsort(coeffs1)[-10:]

print(f'Самые токсичные слова по логистической регрессии: {vectorizer.get_feature_names_out()[np.argsort(coeffs1)[0][-5:]]}')
print(f'Самые токсичные слова по случайному лесу: {vectorizer.get_feature_names_out()[np.argsort(coeffs2)[-5:]]}')


# vectorizer.get_feature_names_out()[np.argsort(coeffs1)[10:]]

#test.loc[np.argsort(y_probas1, axis=0)[:10][:,0],'comment'].values
#np.argsort(y_probas1, axis=0)[:10][:,0]

Самые токсичные слова по логистической регрессии: ['сука' 'сжечь' 'дебил' 'хохлы' 'хохлов']
Самые токсичные слова по случайному лесу: ['нахуй' 'хохлы' 'хохлов' 'тебе' 'Ты']


In [207]:
np.argsort(coeffs2)[-10:]

array([2773, 1919, 4002, 8000,  673, 7723, 4723, 7164, 7721,  998])